In [2]:
import numpy as np
import cv2

# True params to be estimated:
R_truth, _ = cv2.Rodrigues(np.array([0,0,np.deg2rad(30)]))
T_truth = np.array([0,2,1]).reshape(3,1)

# Create known data:
# world points over the plane z=0 (Notice the transpose)
Pw = np.array([[-1,1,0], [2,1,0], [3,4,0],[2,8,0],[0,5,0]]).T
Pl = R_truth.dot(Pw) + T_truth
Ul = Pl/np.sqrt(np.sum((Pl**2),axis=0))

print(f'True parameters R, T')
print(R_truth)
print(T_truth)
print('-----------------------------')
print(f'World points Pw:')
print(Pw)
print(f'Laser points Pw (Not known):')
print(Pl)
print(f'Directional vectors in laser coords:')
print(Ul)

True parameters R, T
[[ 0.8660254 -0.5        0.       ]
 [ 0.5        0.8660254  0.       ]
 [ 0.         0.         1.       ]]
[[0]
 [2]
 [1]]
-----------------------------
World points Pw:
[[-1  2  3  2  0]
 [ 1  1  4  8  5]
 [ 0  0  0  0  0]]
Laser points Pw (Not known):
[[-1.3660254   1.23205081  0.59807621 -2.26794919 -2.5       ]
 [ 2.3660254   3.8660254   6.96410162  9.92820323  6.33012702]
 [ 1.          1.          1.          1.          1.        ]]
Directional vectors in laser coords:
[[-0.4695354   0.29481902  0.08470246 -0.22163248 -0.36342543]
 [ 0.81325917  0.92510617  0.98628996  0.97022117  0.92021165]
 [ 0.34372377  0.23929128  0.14162487  0.09772374  0.14537017]]


In [ ]:
# recap of known/given data:
k0 = np.sqrt(np.sum(T_truth**2,axis=0))
k0 = k0*np.ones((1,Pw.shape[1]))
#k0 = k0 + np.random.normal(0,1,np.prod(k0.shape)).reshape((k0.shape))
Ul = Pl/np.sqrt(np.sum((Pl**2),axis=0))


def step_1(k, Ul, Pw):
    # compute Pl:
    Pl = k*Ul
    # get dPl and dPw:
    dPl = Pl[:,0:-1] - Pl[:,1:]
    dPw = Pw[:,0:-1] - Pw[:,1:]


    cv2.solve




    # construct regressor matrix:
    eta = dPw[0:2,:].T
    
    mz = np.zeros_like(eta)
    M1 = np.column_stack((eta,mz,mz))
    M2 = np.column_stack((mz,eta,mz))
    M3 = np.column_stack((mz,mz,eta))
    M = np.row_stack((M1,M2,M3))

    b = np.row_stack((dPl[0,:].reshape(-1,1), dPl[1,:].reshape(-1,1), dPl[2,:].reshape(-1,1)))
    rs_hat = np.linalg.inv(M.T.dot(M)).dot(M.T.dot(b))
    rs_hat = rs_hat.reshape(3,2)
    rs_hat = rs_hat/np.sqrt(np.sum(rs_hat**2,0,keepdims=True))

    # recovering the rx3 that cannot be estimated:
    rs_mis = np.sqrt(1 - np.sum(rs_hat**2,1,keepdims=True))
    rx3 = rs_mis.reshape(3,1)
    R_hat = np.column_stack((rs_hat,rs_mis))
    a=2
    
step_1(k0,Ul,Pw)



print(k0.shape)

C:\Users\evert\AppData\Local\Temp\ipykernel_440\2009906348.py:29: RuntimeWarning: invalid value encountered in sqrt
  rs_mis = np.sqrt(1 - np.sum(rs_hat**2,1,keepdims=True))


KeyboardInterrupt: 